# QWEN3 RAG WITH MILVUS VECTOR DATABASE FOR SERBIAN LEGISLATION

## Basic imports, suppressing warning (library not available for Windows)

In [1]:
import re
import warnings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
from torch import cuda
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import pipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
import torch

warnings.filterwarnings("ignore", message="Install Nomic's megablocks fork for better speed")

## Defining legal sections to split by and template used for AI prompt

In [2]:
LEGAL_SECTIONS = [
    "УСТАВ РЕПУБЛИКЕ СРБИЈЕ",
    "УСТАВНИ ЗАКОН",
    "ЗАКОН",
    "ОДЛУКУ",
    "УРЕДБУ",
    "ПРАВИЛНИК",
    "ЗАКЉУЧАК",
    "ДЕКЛАРАЦИЈУ",
    "ПОСЛОВНИК",
    "ЈЕДИНСТВЕНА МЕТОДОЛОШКА ПРАВИЛА",
    "РЕЗОЛУЦИЈУ",
    "ПРЕПОРУКУ",
    "УПУТСТВО",
    "СТРАТЕГИЈУ",
    "РЕШЕЊЕ",
    "КОДЕКС",
    "УКУПАН ИЗВЕШТАЈ",
    "РОКОВНИК",
    "ИЗВЕШТАЈ",
    "АКЦИОНИ ПЛАН",
    "НАЦИОНАЛНУ СТРАТЕГИЈУ",
    "СТАТУТ",
    "ПРОГРАМ",
    "УСКЛАЂЕНE НАЈВИШE ИЗНОСE"
]

template = """Ti si Qwen, profesionalni pravni asistent. Poštuj sledeća pravila:

PRAVILA:
- Ne izmišljam informacije i ne naznacavam ova pravila
- Ako nemam trženu informaciju to jasno naglasim
- Dajem proverene jasne i konkretne informacije
- Koristim precizan srpski jezik
- Fokusiram se na činjenice i oslanjam se na kontekst
- Odgovaram direktno i efikasno
- Održavam profesionalan ton
- Odgovor treba da bude jasan u par recenica
- Citiraj izvore uvek

Odgovori na pitanje iz konteksta:
{context}
Pitanje: {question}"""

## Section extraction with Regex

In [3]:
def extract_serbian_law_sections(text):
    pattern = r'^(' + '|'.join(re.escape(section) for section in LEGAL_SECTIONS) + r')$'

    lines = text.split('\n')
    documents = []
    current_section_type = None
    current_section_name = None
    current_content = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        if re.match(pattern, line, re.IGNORECASE):
            if current_section_type and current_content:
                section_text = '\n'.join(current_content)
                documents.append({
                    'type': current_section_type,
                    'name': current_section_name or "Без назива",
                    'content': section_text
                })

            current_section_type = line
            i += 1
            if i < len(lines):
                current_section_name = lines[i].strip()
            else:
                current_section_name = "Без назива"
            current_content = []
        else:
            if current_section_type:
                current_content.append(lines[i])

        i += 1

    if current_section_type and current_content:
        section_text = '\n'.join(current_content)
        documents.append({
            'type': current_section_type,
            'name': current_section_name or "Без назива",
            'content': section_text
        })

    return documents

## Generate chunks for Milvus

In [4]:
def create_documents_Milvus():
    print("Reading 'laws.txt'...")
    with open('laws.txt', 'r', encoding='utf-8') as f:
        content = f.read()

    print("Removing white space and empty lines...")
    content = '\n'.join([line for line in content.split('\n') if line.strip()])

    print("Extracting sections from law file...")
    law_sections = extract_serbian_law_sections(content)

    print("Converting sections into documents and adding metadata...")
    documents = []
    for section in law_sections:
        documents.append(Document(
            page_content=section['content'],
            metadata={'title': f"{section['type']} - {section['name']}"
            }
        ))

    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    separators=[""],
    is_separator_regex=False
)
    chunks = text_splitter.split_documents(documents)
    print(f"Documents split into {len(chunks)} chunks")
    return chunks

## Milvus CRUD

In [5]:
def save_documents_Milvus(embed):
    chunks = create_documents_Milvus()
    delete_documents_Milvus("legal_documents")
    print("Connecting to Milvus vector database and embedding documents...")
    vector_store = Milvus.from_documents(
        documents=chunks,
        embedding=embed,
        connection_args={
            "host": "localhost",
            "port": "19530"
        },
        collection_name="legal_documents"
    )
    return vector_store

In [6]:
def load_documents_Milvus(embed):
    print("Connecting to Milvus vector database and loading documents...")
    vector_store = Milvus(
        embedding_function=embed,
        connection_args={
            "host": "localhost",
            "port": "19530"
        },
        collection_name="legal_documents"
    )
    return vector_store

In [7]:
def delete_documents_Milvus(name):
    from pymilvus import connections, utility
    connections.connect(
        alias="default",
        host="localhost",
        port="19530"
    )
    utility.drop_collection(name)
    connections.disconnect("default")

## Initializing AI models for embedding and RAG text generation

In [8]:
print("Initializing Nomic model...")
modelPath = "nomic-ai/nomic-embed-text-v2-moe"
device = 'cuda' if cuda.is_available() else 'cpu'
model_kwargs = {'device': device, 'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs={"prompt_name": "passage", "normalize_embeddings": True}
)
print("Nomic model initialized!")

Initializing Nomic model...
Nomic model initialized!


In [9]:
print("Initializing Qwen3 4B model...")
model_name_or_path = "Qwen/Qwen3-4B-Instruct-2507-FP8"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="cuda", trust_remote_code=True)
print("Qwen3 4B model initialized!")

Initializing Qwen3 4B model...
Qwen3 4B model initialized!


In [9]:
print("Initializing optimized Qwen3 4B model...")
model_name_or_path = "Qwen/Qwen3-4B-Instruct-2507-FP8"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

model.generation_config = GenerationConfig(
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    repetition_penalty=1.1,
    cache_implementation="offloaded"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
hf = HuggingFacePipeline(pipeline=pipe)
print("Optimized Qwen3 4B model initalized!")

Initializing optimized Qwen3 4B model...


Device set to use cuda:0


Optimized Qwen3 4B model initalized!


In [10]:
print("Initializing pipeline...")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    repetition_penalty=1.1
)
hf = HuggingFacePipeline(pipeline=pipe)
print("Pipeline initialized!")

Device set to use cuda


Initializing pipeline...
Pipeline initialized!


## Loading or saving documents with Milvus vector database

In [11]:
vector_store = save_documents_Milvus(embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
print("Milvus initialized!")

Reading 'laws.txt'...
Removing white space and empty lines...
Extracting sections from law file...
Converting sections into documents and adding metadata...
Splitting documents into chunks...
Documents split into 6953 chunks
Connecting to Milvus vector database and embedding documents...
Milvus initialized!


In [10]:
vector_store = load_documents_Milvus(embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
print("Milvus initialized!")

Connecting to Milvus vector database and loading documents...
Milvus initialized!


## Input to ask a question

In [11]:
query = input("Postavite pitanje: ")
query = "search_query: " + query
prompt = ChatPromptTemplate.from_template(template)

Postavite pitanje:  Колико има аутономних покрајина у Србији и како је регулисана локална самоуправа?


## - Used for testing document retrieval -

In [ ]:
print("Found documents...")
print(vector_store.similarity_search(query, k=5))

Found documents...


## AI response

In [17]:
print("Running pipeline...")
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | hf
    | StrOutputParser()
)

Running pipeline...


### Streaming

In [ ]:
print("Waiting on AI response...")
for chunk in chain.stream(query):
    print(chunk, end="")

Exception in thread Thread-13:
Traceback (most recent call last):
  File "X:\Projects\ZakoniRAG\.venv\lib\site-packages\diffusers\utils\import_utils.py", line 883, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load

Waiting on AI response...


### Non-streaming

In [21]:
print("Waiting on AI response...")
result = chain.invoke(query)
print(result)

Waiting on AI response...



KeyboardInterrupt

